# Mohammad Shafkat Islam
#### Ph.D. Candidate and Research Assistant
#### The University of Iowa
#### E-mail: mohammadshafkat-islam@uiowa.edu
#### Phone: 319-237-5406
#### LinkedIn: https://www.linkedin.com/in/mshafkat

# Python Scraping from Wikipedia:

In [7]:

"""
Step0: Import Necessary Functions.
"""
import pandas as pd
import wikipedia as wp


"""
Step1: Read the html table from wikipedia and find DataFrame object: We read HTML tables into a list of DataFrame objects. It finds the table element, does the parsing and creates a DataFrame.
"""
#The required column names for each of the information extracted from wikipedia page:    
column_names= ["2018 Rank", "City", "State", "2018 estimate","2010 Census", "Change","2016 land area (sq mile)","2016 land area(sq km)","2016 population density (per sq mile)","2016 population density (per sq km)", "location"]    

# Read the html table and find the DataFrame object related to the list of cities.
html = wp.page("List of United States cities by population").html().encode("utf-8")


"""
Step2: Cleaning the data:
    a) Remove the first row since the headers has been manually generated.
    b) Remove the [] which can be found in some cities and states, they are links to the corresponding cities. For our purposes we may not need them.
    c) Convert the values having the corresponding units to simply flaoting point numbers. We may want to process the values.
"""
    
us_cities_df = pd.read_html(html, skiprows=1)[4]
us_cities_df = us_cities_df.replace(to_replace ='\[.*', value = '', regex = True)


list_of_columns_to_be_cleaned = [us_cities_df[6], us_cities_df[7], us_cities_df[8], us_cities_df[9]]
length_of_characters_to_remove = [6,4,6,4]

character_index = 0
for current_column in list_of_columns_to_be_cleaned:
    
    processed_column = current_column
    for i in range (0 ,len(processed_column)):
        
        length_of_current_column_character_to_remove = length_of_characters_to_remove[character_index]
        #print(length_of_current_column_character_to_remove)
        x= str(processed_column[i][:-length_of_current_column_character_to_remove])
        processed_column[i]= x
    character_index +=1
    
"""
Step4: Scrape data from additional sources: I have written this poriton of the script which extracts additional information.
The goal of this portion is to extrac the crime rate for the most populus cities and combine the information with the previous portion.
"""    
  

# # Read the html table and find the DataFrame object related to the list of cities.
# crime_rate_html = wp.page("List of United States cities by crime rate").html().encode("utf-8")
# crime_rate_df = pd.read_html(crime_rate_html, skiprows=1)[0]    
# crime_rate_df.head()


"""
Step5: Write results into csv files.
"""
us_cities_df.columns = column_names
us_cities_df.to_csv('beautifulsoup_pandas_new.csv', index=False, encoding = "utf-8")
us_cities_df.head()

/local/vol00/scratch/Shafkat/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,2018 Rank,City,State,2018 estimate,2010 Census,Change,2016 land area (sq mile),2016 land area(sq km),2016 population density (per sq mile),2016 population density (per sq km),location
0,1,New York City,New York,8398748,8175133,+2.74%,301.5,780.9,"28,317","10,933",40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W﻿...
1,2,Los Angeles,California,3990456,3792621,+5.22%,468.7,"1,213.9","8,484","3,276",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°...
2,3,Chicago,Illinois,2705994,2695598,+0.39%,227.3,588.7,"11,900","4,600",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W﻿...
3,4,Houston,Texas,2325502,2100263,+10.72%,637.5,"1,651.1","3,613","1,395",29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W﻿...
4,5,Phoenix,Arizona,1660272,1445632,+14.85%,517.6,"1,340.6","3,120","1,200",33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°...


In [8]:
# Read the html table and find the DataFrame object related to the list of cities.
crime_column_names = ["State", "City", "Population","Total Violent Crime", "Murder and Nonnegligent manslaughter",
                      "Rape","Robbery","Aggravated assault","Total Property Crime",
                      "Burglary","Larceny-theft","Motor vehicle theft","Arson"]    

crime_rate_html = wp.page("List of United States cities by crime rate").html().encode("utf-8")
crime_rate_df = pd.read_html(crime_rate_html, skiprows=2)[0]
crime_rate_df[1] = crime_rate_df[1].replace(to_replace =r'[0-9]', value = '', regex = True)

crime_rate_df.columns = crime_column_names
crime_rate_df.head()
merged_df = us_cities_df.merge(crime_rate_df, how = 'inner', on = ['City', 'State'])
merged_df

,2018 Rank,City,State,2018 estimate,2010 Census,Change,2016 land area (sq mile),2016 land area(sq km),2016 population density (per sq mile),2016 population density (per sq km),...,Total Violent Crime,Murder and Nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Total Property Crime,Burglary,Larceny-theft,Motor vehicle theft,Arson
0,2,Los Angeles,California,3990456,3792621,+5.22%,468.7,"1,213.9","8,484","3,276",...,761.31,7.01,61.27,269.87,423.17,2535.92,415.96,1640.99,478.97,35.29
1,3,Chicago,Illinois,2705994,2695598,+0.39%,227.3,588.7,"11,900","4,600",...,1098.86,24.13,65.11,439.26,570.36,3263.80,477.13,2358.83,427.84,18.99
2,4,Houston,Texas,2325502,2100263,+10.72%,637.5,"1,651.1","3,613","1,395",...,1095.23,11.50,58.42,417.96,607.34,4128.41,731.66,2900.82,495.93,28.44
3,5,Phoenix,Arizona,1660272,1445632,+14.85%,517.6,"1,340.6","3,120","1,200",...,760.93,9.55,69.46,200.28,481.64,3670.71,778.57,2426.69,465.46,11.56
4,6,Philadelphia,Pennsylvania,1584138,1526006,+3.81%,134.2,347.6,"11,683","4,511",...,947.58,20.06,75.02,382.46,470.04,3063.48,418.25,2297.23,348.00,26.21
5,7,San Antonio,Texas,1532233,1327407,+15.43%,461.0,"1,194.0","3,238","1,250",...,707.50,8.15,83.51,151.11,464.72,4844.84,770.82,3622.65,451.37,13.61
6,8,San Diego,California,1425976,1307402,+9.07%,325.2,842.3,"4,325","1,670",...,366.61,2.46,39.25,99.01,225.89,1842.97,268.03,1214.37,360.57,11.09
7,9,Dallas,Texas,1345047,1197816,+12.29%,340.9,882.9,"3,866","1,493",...,774.64,12.48,62.08,327.00,373.09,3185.09,737.66,1856.26,591.16,26.22
8,10,San Jose,California,1030119,945942,+8.90%,177.5,459.7,"5,777","2,231",...,403.65,3.08,55.03,132.62,212.91,2440.70,378.40,1284.69,777.62,16.67
9,11,Austin,Texas,964254,790390,+22.00%,312.7,809.9,"3,031","1,170",...,414.84,2.57,85.81,101.55,224.91,3189.57,450.64,2525.03,213.90,7.82
